In [1]:
#Import libraries
import numpy as np
import mediapipe as mp 
import cv2

In [2]:
#Initialize MediaPipe pose and drawing utilities
mp_pose = mp.solutions.pose
mp_drawings = mp.solutions.drawing_utils

In [5]:
#Function to calculate the angle between three points
def calc_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    # Calculate the vectors
    ab = a - b
    cb = c - b
    
    # Calculate the dot product and magnitude of the vectors
    dot_product = np.dot(ab, cb)
    magnitude_ab = np.linalg.norm(ab)
    magnitude_cb = np.linalg.norm(cb)
    
    # Calculate the cosine of the angle
    cos_angle = dot_product / (magnitude_ab * magnitude_cb)
    
    # Clip the cosine value to avoid numerical errors that might occur outside the [-1, 1] range
    cos_angle = np.clip(cos_angle, -1.0, 1.0)
    
    # Calculate the angle in radians and then convert to degrees
    angle = np.arccos(cos_angle)
    angle = np.degrees(angle)
    
    return angle

In [6]:
#Initialize counter and state tracking
counter = 0
previous_position = "Unknown"

# Cell 5: Capture video from webcam
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        frame = cv2.flip(frame, 1)
        image_height, image_width, _ = frame.shape
        rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb_image.flags.writeable = False
        
        results = pose.process(rgb_image)
        rgb_image.flags.writeable = True
        bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
        
        if results.pose_landmarks:
            mp_drawings.draw_landmarks(
                bgr_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawings.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                mp_drawings.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Extract coordinates for shoulder, elbow, and hip and convert to pixel values for left hand
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x * image_width, 
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y * image_height]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x * image_width, 
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y * image_height]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * image_width, 
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * image_height]
            
            # Extract coordinates for shoulder, elbow, and hip and convert to pixel values for right hand
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x * image_width, 
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y * image_height]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x * image_width, 
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y * image_height]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * image_width, 
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * image_height]
            
            # Calculate angles
            left_angle = calc_angle(left_shoulder, left_elbow, left_hip)
            right_angle = calc_angle(right_shoulder, right_elbow, right_hip)
            
            # Check if both angles are around 180 degrees
            if left_angle < 20 and right_angle < 20 and previous_position != "Up":
                counter += 1
            
            # Update the previous position
            previous_position = "Up" if left_angle < 20 and right_angle < 20 else "Not Up"
            
            # Display the angles and counter on the frame
            cv2.putText(bgr_image, f"Left Angle: {int(left_angle)}", (10, 30),
                        cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(bgr_image, f"Right Angle: {int(right_angle)}", (10, 60),
                        cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(bgr_image, f"Counter: {counter}", (10, 90),
                        cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
        
        cv2.imshow("Camera", bgr_image)
        
        key = cv2.waitKey(10) & 0xFF
        if key == ord('q') or key == 27 or key == ord('x') or key == ord('Q') or key == ord('X'):
            break

cap.release()
cv2.destroyAllWindows()

In [7]:
#Print the final value of counter
print("The total number of reps for the exercise are: ",counter)

The total number of reps for the exercise are:  0
